<a href="https://colab.research.google.com/github/kktsuji/train-lora-cell-rois/blob/main/train_lora_cell_rois.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Variable Settings

# RESOLUTION = "40" # original size
# RESOLUTION = "256"
RESOLUTION = "512" # recommended for sd-1.x

NUM_TRAIN_EPOCHS = "300"

out_dir_name = f"cell-rois_lora_resolution-{RESOLUTION}_epochs-{NUM_TRAIN_EPOCHS}"

In [ ]:
import os

out_dir_path = f"/content/drive/MyDrive/Research/LoRAs/{out_dir_name}/"
log_dir_path = f"/content/drive/MyDrive/Research/LoRAs/{out_dir_name}/logs/"
if not os.path.exists(out_dir_path):
  os.makedirs(out_dir_path)
if not os.path.exists(log_dir_path):
  os.makedirs(log_dir_path)
print(out_dir_path)
print(log_dir_path)

base_data_dir_path = "/content/drive/MyDrive/Research/Data/250606_cell_rois/pseudo_rgb/"
print(base_data_dir_path)

model_id = "CompVis/stable-diffusion-v1-4"
model_dir_path = "/content/drive/MyDrive/Research/Models/" + model_id
print(model_dir_path)

if (os.path.exists(out_dir_path) or
  os.path.exists(log_dir_path) or
  os.path.exists(base_data_dir_path) or
  os.path.exists(model_dir_path)):
  print("All directories exist")
else:
  from google.colab import runtime
  runtime.unassign()
  raise Exception("Directory not found")

In [ ]:
if not os.path.exists(model_dir_path):
  from diffusers import StableDiffusionPipeline
  import torch
  os.makedirs(model_dir_path)
  pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32)
  pipe.save_pretrained(model_dir_path)

In [ ]:
!git clone -b develop https://github.com/kktsuji/peft.git

import sys
sys.path.append('./peft/examples/stable_diffusion')
import train_dreambooth

print(train_dreambooth.UNET_TARGET_MODULES)

# for "CompVis/stable-diffusion-v1-4"
unet_target_modules = [
    # Convolution layers
    "conv_in", "conv1", "conv2", "conv_out",
    "conv_shortcut", "conv",
    # Linear layers
    "to_q", "to_k", "to_v", "to_out.0",
    "linear_1", "linear_2", "proj_in", "proj_out", "proj",
    "time_emb_proj", "ff.net.0.proj", "ff.net.2",
]

train_dreambooth.UNET_TARGET_MODULES = unet_target_modules
print(train_dreambooth.UNET_TARGET_MODULES)

In [ ]:
input_args = [
    "--pretrained_model_name_or_path", model_dir_path,
    "--instance_data_dir", base_data_dir_path,
    "--instance_prompt", "rds",
    "--seed", "0",
    "--resolution", RESOLUTION,
    "--output_dir", out_dir_path,
    "--num_train_epochs", NUM_TRAIN_EPOCHS,
    "--lr_scheduler", "cosine",
    "--lr_warmup_steps", "5",
    "--learning_rate", "1e-4",
    "--logging_dir", "logs",
    "--report_to", "tensorboard",
    "lora",
    "--unet_r", "16",
    "--unet_alpha", "16"
]

args = train_dreambooth.parse_args(input_args)
train_dreambooth.main(args)

In [ ]:
from google.colab import runtime
runtime.unassign()